# HomeWork 2 - Working with Data In Pandas - Stock Market Analysis

#0) Imports and Installs

In [1]:
!pip install yfinance pyarrow fastparquet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 13.2 MB/s eta 0:00:00


In [2]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

# for graphs
import matplotlib.pyplot as plt

# for Data ingestion and manipulation
import pandas as pd
import requests



### Question 1: IPO Filings Web Scraping and Data Processing

**What's the total sum ($m) of 2023 filings that happenned of Fridays?**

Re-use the [Code Snippet 1] example to get the data from web for this endpoint: https://stockanalysis.com/ipos/filings/
Convert the 'Filing Date' to datetime(), 'Shares Offered' to float64 (if '-' is encountered, populate with NaNs).
Define a new field 'Avg_price' based on the "Price Range", which equals to NaN if no price is specified, to the price (if only one number is provided), or to the average of 2 prices (if a range is given).

You may be inspired by the function `extract_numbers()` in [Code Snippet 4], or you can write your own function to "parse" a string.
Define a column "Shares_offered_value", which equals to "Shares Offered" * "Avg_price" (when both columns are defined; otherwise, it's NaN)

In [29]:
import pandas as pd
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/filings/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)
ipo_dfs

[      Filing Date Symbol                           Company Name  \
 0     May 3, 2024    TBN         Tamboran Resources Corporation   
 1    Apr 29, 2024   HWEC                   HW Electro Co., Ltd.   
 2    Apr 29, 2024   DTSQ  DT Cloud Star Acquisition Corporation   
 3    Apr 26, 2024   EURK                Eureka Acquisition Corp   
 4    Apr 26, 2024    HDL    Super Hi International Holding Ltd.   
 ..            ...    ...                                    ...   
 320  Jan 21, 2020   GOXS                            Goxus, Inc.   
 321  Jan 21, 2020   UTXO                 UTXO Acquisition, Inc.   
 322   Dec 9, 2019   LOHA                           Loha Co. Ltd   
 323   Oct 4, 2019   ZGHB  China Eco-Materials Group Co. Limited   
 324  Dec 27, 2018   FBOX              Fit Boxx Holdings Limited   
 
         Price Range Shares Offered  
 0                 -              -  
 1             $3.00        3750000  
 2            $10.00        6000000  
 3            $10.00        50

In [30]:
# Extract the dataframe from the list
ipos_filings = ipo_dfs[0]
ipos_filings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325 entries, 0 to 324
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Filing Date     325 non-null    object
 1   Symbol          325 non-null    object
 2   Company Name    325 non-null    object
 3   Price Range     325 non-null    object
 4   Shares Offered  325 non-null    object
dtypes: object(5)
memory usage: 12.8+ KB


Check if there are null values

In [31]:
# simple way of checking NULLs
# (you need to understand how vector operations work .isnull() and calls chaining .isnull().sum())
ipos_filings.isnull().sum()

Filing Date       0
Symbol            0
Company Name      0
Price Range       0
Shares Offered    0
dtype: int64

Functions for transformation and feture engineering

In [32]:
def get_avg_prices(input_string):
  # Extract all the values in the string
  ranges = input_string.split('-')
  # Convert them into float
  values=[float(v.strip()[1:]) for v in ranges if len(v)>0]
  # Return the mean or nan if there are no numbers
  return np.mean(values)

In [33]:
def shares_value(row):
    return row.shares_offered*row.avg_price

Create the transformed dataset

In [35]:
# Apply all the transformations
df_ipos= (
    ipos_filings
    .rename(columns={'Price Range': 'price_range', 'Company Name': 'company_name'})
    .assign(filing_date=lambda x: pd.to_datetime(x['Filing Date'], format='mixed'))
    .assign(shares_offered=lambda x: pd.to_numeric(x['Shares Offered'], errors='coerce'))
    .assign(avg_price=lambda df: df.price_range.map(lambda x: get_avg_prices(x)))
    .assign(shares_offered_value=lambda df: df.apply(shares_value, axis="columns"))
    .drop(columns=['Filing Date', 'Shares Offered', ])
)
df_ipos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325 entries, 0 to 324
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Symbol                325 non-null    object        
 1   company_name          325 non-null    object        
 2   price_range           325 non-null    object        
 3   filing_date           325 non-null    datetime64[ns]
 4   shares_offered        252 non-null    float64       
 5   avg_price             258 non-null    float64       
 6   shares_offered_value  249 non-null    float64       
dtypes: datetime64[ns](1), float64(3), object(3)
memory usage: 17.9+ KB


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Find the total sum in $m (millions of USD, closest INTEGER number) for all fillings during 2023, which happened on Fridays (`Date.dt.dayofweek()==4`). You should see 32 records in total, 24 of it is not null.

(additional: you can read about [S-1 IPO filing](https://www.dfinsolutions.com/knowledge-hub/thought-leadership/knowledge-resources/what-s-1-ipo-filing) to understand the context)


In [36]:
df_ipos

,Symbol,company_name,price_range,filing_date,shares_offered,avg_price,shares_offered_value
0,TBN,Tamboran Resources Corporation,-,2024-05-03,NaN,NaN,NaN
1,HWEC,"HW Electro Co., Ltd.",$3.00,2024-04-29,3750000.0,3.00,11250000.0
2,DTSQ,DT Cloud Star Acquisition Corporation,$10.00,2024-04-29,6000000.0,10.00,60000000.0
3,EURK,Eureka Acquisition Corp,$10.00,2024-04-26,5000000.0,10.00,50000000.0
4,HDL,Super Hi International Holding Ltd.,-,2024-04-26,NaN,NaN,NaN
...,...,...,...,...,...,...,...
320,GOXS,"Goxus, Inc.",$8.00 - $10.00,2020-01-21,1500000.0,9.00,13500000.0
321,UTXO,"UTXO Acquisition, Inc.",$10.00,2020-01-21,5000000.0,10.00,50000000.0
322,LOHA,Loha Co. Ltd,$8.00 - $10.00,2019-12-09,2500000.0,9.00,22500000.0
323,ZGHB,China Eco-Materials Group Co. Limited,$4.00,2019-10-04,4300000.0,4.00,17200000.0


In [37]:
# Calculate the sum in million dolars and round it to the closest integer
np.rint(np.sum(
    df_ipos
    .loc[lambda df: (df.filing_date.dt.dayofweek == 4) & (df.filing_date.dt.year == 2023)]
    ['shares_offered_value']/1000000
))

286.0

### Question 2:  IPOs "Fixed days hold" strategy


**Find the optimal number of days X (between 1 and 30), where 75% quantile growth is the highest?**


Reuse [Code Snippet 1] to retrieve the list of IPOs from 2023 and 2024 (from URLs: https://stockanalysis.com/ipos/2023/ and https://stockanalysis.com/ipos/2024/).
Get all OHLCV daily prices for all stocks with an "IPO date" before March 1, 2024 ("< 2024-03-01") - 184 tickers (without 'RYZB'). Please remove 'RYZB', as it is no longer available on Yahoo Finance.

Sometimes you may need to adjust the symbol name (e.g., 'IBAC' on stockanalysis.com -> 'IBACU' on Yahoo Finance) to locate OHLCV prices for all stocks.
Some of the tickers like 'DYCQ' and 'LEGT' were on the market less than 30 days (11 and 21 days, respectively). Let's leave them in the dataset; it just means that you couldn't hold them for more days than they were listed.


Read IPOS from 2023 and 2024, concatenate and filter based on IPO Date

In [38]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/2023/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)
ipos_2023 = ipo_dfs[0]

url = "https://stockanalysis.com/ipos/2024/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)
ipos_2024 = ipo_dfs[0]
# Check the length of the datasets
print(len(ipos_2023))
print(len(ipos_2024))
# Concatenate the dataset
ipos=pd.concat([ipos_2024,ipos_2023], axis=0)
len(ipos)

154
64


218

In [ ]:
ipos_2023[ipos_2023['Symbol']=='AACT']

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
96,"Apr 21, 2023",AACT,Ares Acquisition Corporation II,$10.00,$10.62,6.15%


Extract the symbols we need to analyze

In [40]:
# Extract the list of symbols
ipos_2024_03_01=(
    ipos
    .assign(ipo_date=lambda x: pd.to_datetime(x['IPO Date'], format='mixed'))
    .loc[lambda df: df.ipo_date <'2024-03-01']
    .loc[lambda df: df.Symbol !='RYZB']
)

symbols = (
    ipos_2024_03_01
    ['Symbol']
    .values
    .tolist()
)
# convert to a single string
print("Count of symbols: ", len(symbols))
symbols_string = " ".join(symbols)
print(symbols_string)

Count of symbols:  184
SMXT VHAI DYCQ CHRO UMAC TBBB MGX HLXB TELO KYTX PMNT AHR LEGT ANRO GUTS AS FBLG BTSG AVBP HAO CGON YIBO SUGP JL KSPI JVSA PSBD CCTG SYNX SDHC ROMA IROH LGCB ZKH BAYA INHD AFJK GSIW FEBO CLBR ELAB RR DDC SHIM GLAC SGN HG CRGX ANSC AITR GVH LXEO PAPL ATGL MNR WBUY NCL BIRK GMM PMEC LRHC GPAK SPKL QETA MSS ANL SYRA VSME LRE TURB MDBH KVYO CART DTCK NMRA ARM SPPL NWGL SWIN IVP NNAG SRM SPGC LQR NRXS FTEL MIRA PXDT HRYU CTNT SRFM PRZO HYAC KVAC JNVR ELWS WRNT TSBX ODD APGE NETD SGMT BOWN SXTP PWM VTMX INTS SVV KGS FIHL GENK BUJA BOF AZTR CAVA ESHA ATMU ATS IPXX CWD SGE SLRN ALCY KVUE GODN TRNR AACT JYD USGO UCAR WLGS TPET TCJH GDTC VCIG GDHG ARBB ISPR MGIH MWG HSHP SFWL SYT HKIT CHSN TBMC HLP ZJYL TMTC YGFGF OAKU BANL OMH MGRX FORL ICG IZM AESI AIXI SBXC BMR DIST GXAI MARX BFRG ENLT MLYS PTHR BLAC NXT HSAI LSDI LICN GPCR ASST CETU TXO BREA GNLX QSG CVKD SKWD ISRL MGOL


Set the start and end date:

In [41]:
start_2023 = date(2023,1,1)
end_2024 = date(2024,3,1)

Download the data for all the tickers

In [42]:
data_ohlcv = yf.download(symbols_string, start="2023-01-01", period="max", interval="1d")

[*********************100%%**********************]  184 of 184 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['PTHR']: Exception('%ticker%: No timezone found, symbol may be delisted')


PTHR symbol is not in yahoo finance library. We need to search for its symbol in yfinance. Let's watch its company name

In [43]:
ipos_2024_03_01[ipos_2024_03_01['Symbol']=='PTHR']

,IPO Date,Symbol,Company Name,IPO Price,Current,Return,ipo_date
137,"Feb 10, 2023",PTHR,"Pono Capital Three, Inc.",$10.00,$5.72,-42.80%,2023-02-10


Some symbol are not present in the OHLCV data for the whole date range, for example DYCQ

In [44]:
dycq= yf.Ticker("DYCQ")
df_dycq = dycq.history(period="max", interval="1d")
df_dycq

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2024-04-12 00:00:00-04:00,10.090,10.110,10.070,10.100,454200,0.0,0.0
2024-04-15 00:00:00-04:00,10.100,10.140,10.100,10.115,465500,0.0,0.0
2024-04-16 00:00:00-04:00,10.110,10.120,10.100,10.120,1603800,0.0,0.0
2024-04-17 00:00:00-04:00,10.110,10.120,10.110,10.120,29500,0.0,0.0
2024-04-18 00:00:00-04:00,10.120,10.130,10.120,10.120,19900,0.0,0.0
2024-04-19 00:00:00-04:00,10.120,10.120,10.110,10.110,62100,0.0,0.0
2024-04-22 00:00:00-04:00,10.120,10.130,10.120,10.120,61700,0.0,0.0
2024-04-23 00:00:00-04:00,10.125,10.130,10.120,10.120,40000,0.0,0.0
2024-04-24 00:00:00-04:00,10.127,10.150,10.120,10.120,224900,0.0,0.0


PTHR is not in yfinance library. Googling we can get the yfinance for ""Pono Capital" (PHTR in stockanalysiscom) is `PTHRU`. Let's try to download it to confirm that it is te right symbol.

In [ ]:
pthru_ohlcv = yf.download("PTHRU", start="2023-01-01", period="max", interval="1d")
pthru_ohlcv.info()

[*********************100%%**********************]  1 of 1 completed

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 232 entries, 2023-02-10 to 2024-01-12
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       232 non-null    float64
 1   High       232 non-null    float64
 2   Low        232 non-null    float64
 3   Close      232 non-null    float64
 4   Adj Close  232 non-null    float64
 5   Volume     232 non-null    int64  
dtypes: float64(5), int64(1)
memory usage: 12.7 KB


We replace the symbol `PTHR` with `PTHRU`:

In [45]:
symbol_lst = symbols_string.split()
print(symbol_lst)
print(len(symbol_lst))
symbol_lst[symbol_lst.index('PTHR')]='PTHRU'
print(symbol_lst)
print(len(symbol_lst))
symbols_string = " ".join(symbol_lst)
print(len(symbols_string))
#symbols_string +=" PTHRU"

['SMXT', 'VHAI', 'DYCQ', 'CHRO', 'UMAC', 'TBBB', 'MGX', 'HLXB', 'TELO', 'KYTX', 'PMNT', 'AHR', 'LEGT', 'ANRO', 'GUTS', 'AS', 'FBLG', 'BTSG', 'AVBP', 'HAO', 'CGON', 'YIBO', 'SUGP', 'JL', 'KSPI', 'JVSA', 'PSBD', 'CCTG', 'SYNX', 'SDHC', 'ROMA', 'IROH', 'LGCB', 'ZKH', 'BAYA', 'INHD', 'AFJK', 'GSIW', 'FEBO', 'CLBR', 'ELAB', 'RR', 'DDC', 'SHIM', 'GLAC', 'SGN', 'HG', 'CRGX', 'ANSC', 'AITR', 'GVH', 'LXEO', 'PAPL', 'ATGL', 'MNR', 'WBUY', 'NCL', 'BIRK', 'GMM', 'PMEC', 'LRHC', 'GPAK', 'SPKL', 'QETA', 'MSS', 'ANL', 'SYRA', 'VSME', 'LRE', 'TURB', 'MDBH', 'KVYO', 'CART', 'DTCK', 'NMRA', 'ARM', 'SPPL', 'NWGL', 'SWIN', 'IVP', 'NNAG', 'SRM', 'SPGC', 'LQR', 'NRXS', 'FTEL', 'MIRA', 'PXDT', 'HRYU', 'CTNT', 'SRFM', 'PRZO', 'HYAC', 'KVAC', 'JNVR', 'ELWS', 'WRNT', 'TSBX', 'ODD', 'APGE', 'NETD', 'SGMT', 'BOWN', 'SXTP', 'PWM', 'VTMX', 'INTS', 'SVV', 'KGS', 'FIHL', 'GENK', 'BUJA', 'BOF', 'AZTR', 'CAVA', 'ESHA', 'ATMU', 'ATS', 'IPXX', 'CWD', 'SGE', 'SLRN', 'ALCY', 'KVUE', 'GODN', 'TRNR', 'AACT', 'JYD', 'USGO', '

Download the data with the right symbols

In [ ]:
data_ohlcv = yf.download(symbols_string, start="2023-01-01", period="max", interval="1d")
data_ohlcv.info()


[*********************100%%**********************]  184 of 184 completed


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 332 entries, 2023-01-03 to 2024-04-29
Columns: 1104 entries, ('Adj Close', 'AACT') to ('Volume', 'ZKH')
dtypes: float64(1102), int64(2)
memory usage: 2.8 MB


In [46]:
#earnings = data_ohlcv['Adj Close'] - data_ohlcv['Adj Close'].shift(-1)
earnings = data_ohlcv['Adj Close'].shift(-1) - data_ohlcv['Adj Close']
earnings.head()

Ticker,AACT,AESI,AFJK,AHR,AITR,AIXI,ALCY,ANL,ANRO,ANSC,...,VHAI,VSME,VTMX,WBUY,WLGS,WRNT,YGFGF,YIBO,ZJYL,ZKH
Date,,,,,,,,,,,,,,,,,,,,,
2023-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Generate 30 additional columns: growth_future_1d ... growth_future_30d

In [47]:
# FUTURE Growth
#data_ohlcv.loc[:]['growth_future_1d'][:] = data_ohlcv['Adj Close'] - data_ohlcv['Adj Close'].shift(-1)
for day in range(1,31):
  #earnings = data_ohlcv['Adj Close'] - data_ohlcv['Adj Close'].shift(-day)
  earnings = data_ohlcv['Adj Close'].shift(-day) - data_ohlcv['Adj Close']
  data_ohlcv = pd.concat([data_ohlcv, pd.concat({f'growth_future_{day}': earnings}, axis=1)], axis=1)

data_ohlcv.info()
data_ohlcv.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 338 entries, 2023-01-03 to 2024-05-07
Columns: 6624 entries, ('Adj Close', 'AACT') to ('growth_future_30', 'ZKH')
dtypes: float64(6622), int64(2)
memory usage: 17.1 MB


Adj Close                                             ...  \
Ticker          AACT AESI AFJK AHR AITR AIXI ALCY ANL ANRO ANSC  ...   
Date                                                             ...   
2023-01-03       NaN  NaN  NaN NaN  NaN  NaN  NaN NaN  NaN  NaN  ...   
2023-01-04       NaN  NaN  NaN NaN  NaN  NaN  NaN NaN  NaN  NaN  ...   
2023-01-05       NaN  NaN  NaN NaN  NaN  NaN  NaN NaN  NaN  NaN  ...   
2023-01-06       NaN  NaN  NaN NaN  NaN  NaN  NaN NaN  NaN  NaN  ...   
2023-01-09       NaN  NaN  NaN NaN  NaN  NaN  NaN NaN  NaN  NaN  ...   

           growth_future_30                                               
Ticker                 VHAI VSME VTMX WBUY WLGS WRNT YGFGF YIBO ZJYL ZKH  
Date                                                                      
2023-01-03              NaN  NaN  NaN  NaN  NaN  NaN   NaN  NaN  NaN NaN  
2023-01-04              NaN  NaN  NaN  NaN  NaN  NaN   NaN  NaN  NaN NaN  
2023-01-05              NaN  NaN  NaN  NaN  NaN  NaN   NaN  NaN  NaN NaN  
2023-01-06              NaN  NaN  NaN  NaN  NaN  NaN   NaN  NaN  NaN NaN  
2023-01-09              NaN  NaN  NaN  NaN  NaN  NaN   NaN  NaN  NaN NaN  

[5 rows x 6624 columns]

There 6624 columns, it is right, 36 metrics * 184 symbols.

In [48]:
cols = [f'growth_future_{day}' for day in range(1,31)]

In [49]:
# Get the columns we are interested in
cols = [f'growth_future_{day}' for day in range(1,31)]
# Group by ever future growth column and sum their values
stats_growth =(
    data_ohlcv[cols]
    .groupby(level=0, axis=1).sum()
)

stats_growth.head()

,growth_future_1,growth_future_10,growth_future_11,growth_future_12,growth_future_13,growth_future_14,growth_future_15,growth_future_16,growth_future_17,growth_future_18,...,growth_future_28,growth_future_29,growth_future_3,growth_future_30,growth_future_4,growth_future_5,growth_future_6,growth_future_7,growth_future_8,growth_future_9
Date,,,,,,,,,,,,,,,,,,,,,
2023-01-03,1.310001,7.209999,7.209999,7.299999,7.840001,7.890001,8.170001,8.500000,10.279999,10.739998,...,24.849998,25.749999,2.860000,25.960000,3.720001,3.720001,5.680000,6.869998,5.869998,6.759998
2023-01-04,0.020000,5.899997,5.989997,6.530000,6.579999,6.859999,7.189999,8.969998,9.429997,8.710000,...,24.439998,24.649999,2.409999,25.909999,2.409999,4.369998,5.559997,4.559997,5.449996,5.899997
2023-01-05,1.529999,5.969997,6.510000,6.559999,6.839999,7.169999,8.949998,9.409997,8.690000,8.839997,...,24.629999,25.889999,2.389999,25.239997,4.349998,5.539997,4.539997,5.429996,5.879997,5.879997
2023-01-06,0.860001,4.980001,5.030000,5.310000,5.640000,7.419999,7.879998,7.160001,7.309999,7.250001,...,24.360000,23.709999,2.820000,22.399999,4.009998,3.009998,3.899998,4.349998,4.349998,4.439999
2023-01-09,0.000000,4.170000,4.450000,4.780000,6.559999,7.019998,6.300000,6.449998,6.390000,6.649999,...,22.849998,21.539999,3.149998,21.349998,2.149998,3.039997,3.489998,3.489998,3.579998,4.120000


In [53]:
# Calculate the descriptive statistics
stats = stats_growth.describe()
stats

,growth_future_1,growth_future_10,growth_future_11,growth_future_12,growth_future_13,growth_future_14,growth_future_15,growth_future_16,growth_future_17,growth_future_18,...,growth_future_28,growth_future_29,growth_future_3,growth_future_30,growth_future_4,growth_future_5,growth_future_6,growth_future_7,growth_future_8,growth_future_9
count,338.000000,338.000000,338.000000,338.000000,338.000000,338.000000,338.000000,338.000000,338.000000,338.000000,...,338.000000,338.000000,338.000000,338.000000,338.000000,338.000000,338.000000,338.000000,338.000000,338.000000
mean,-13.936034,-34.960294,-36.704404,-38.556085,-40.398324,-42.223363,-44.220090,-45.878982,-47.448827,-48.961216,...,-60.809735,-61.741645,-19.935484,-64.022606,-22.322778,-24.218386,-26.448140,-28.963081,-31.038622,-32.990226
std,196.623727,236.804845,237.251725,237.877200,238.174584,237.001230,241.700777,244.527464,244.973801,249.330314,...,264.596501,246.143100,217.382814,240.846596,227.275955,223.182744,218.571136,228.274337,232.232421,233.451602
min,-3527.130910,-3915.114210,-3914.355970,-3921.679135,-3911.458526,-3833.367702,-3892.695763,-3909.392409,-3877.635112,-3930.396550,...,-4000.803810,-3545.687675,-3765.865653,-3401.031923,-3919.048592,-3786.114367,-3642.941519,-3804.928568,-3859.069476,-3868.367024
25%,-18.361059,-72.966737,-82.016869,-93.886858,-97.152744,-105.937809,-113.394175,-112.341251,-127.833356,-131.036299,...,-184.665567,-183.881471,-35.488722,-188.260060,-39.486837,-48.444036,-50.593064,-50.076736,-56.775596,-60.357859
50%,-2.877993,-16.369751,-18.883315,-18.096140,-19.303643,-23.533409,-23.678846,-23.418813,-22.404112,-21.556263,...,-15.644781,-13.919650,-5.646460,-14.415953,-7.487176,-8.853319,-10.151505,-13.473924,-15.109306,-17.744099
75%,5.884278,7.207744,7.991655,6.881901,8.386303,6.979998,6.769614,7.949373,7.504622,8.329655,...,15.044162,16.245002,8.910976,16.320539,8.906609,9.739909,10.436519,8.609210,7.955868,7.783556
max,454.085677,510.419668,521.852117,524.532135,503.758516,544.660914,528.548452,506.642616,503.277567,515.114782,...,417.278138,512.024021,583.226213,459.967023,561.320376,592.617524,576.505062,554.599225,548.786335,521.346073


In [54]:
# Get the column with the max value in every statistic
stats.idxmax(axis=1)

count     growth_future_1
mean      growth_future_1
std      growth_future_28
min      growth_future_30
25%       growth_future_1
50%       growth_future_1
75%      growth_future_30
max       growth_future_2
dtype: object

Find X, when the 75% quantile growth (among 185 investments) is the highest. X is the growth_future_30.

### Question 3: Is Growth Concentrated in the Largest Stocks?

**Get the share of days (percentage as int) when Large Stocks outperform (growth_7d - growth over 7 periods back) the Largest stocks?**


Reuse [Code Snippet 5] to obtain OHLCV stats for 33 stocks
for 10 full years of data (2014-01-01 to 2023-12-31):


In [55]:
# GEt the LARGEST STOCK companies
US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM'] #11

EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA'] #11

INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS'] #11

LARGEST_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS
print('Num stocks in LARGEST STOCKS: ',len(LARGEST_STOCKS))

NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL'] #11

NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA'] #12

NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS'] #11

LARGE_STOCKS = NEW_EU + NEW_US + NEW_INDIA
print('Num stocks in LARGE STOCKS: ',len(LARGE_STOCKS))

Num stocks in LARGEST STOCKS:  33
Num stocks in LARGE STOCKS:  34


In [56]:
large_stocks_str = " ".join(LARGE_STOCKS)
large_ohlcv = yf.download(large_stocks_str, period="max", #start="2014-01-01", end="2023-12-31",
                          interval="1d")
large_ohlcv.info()

largest_stocks_str = " ".join(LARGEST_STOCKS)
largest_ohlcv = yf.download(largest_stocks_str, period="max", #start="2014-01-01", end="2023-12-31",
                         interval="1d") #period="max",
largest_ohlcv.info()

[*********************100%%**********************]  33 of 33 completed


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 15964 entries, 1962-01-02 to 2024-05-07
Columns: 198 entries, ('Adj Close', 'ADANIENT.NS') to ('Volume', 'XOM')
dtypes: float64(198)
memory usage: 24.2 MB


[*********************100%%**********************]  33 of 33 completed


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 13356 entries, 1972-06-01 to 2024-05-07
Columns: 198 entries, ('Adj Close', 'AAPL') to ('Volume', 'V')
dtypes: float64(198)
memory usage: 20.3 MB


In [57]:
# Keep only the Adj Close
large_ohlcv =large_ohlcv.loc[:, ['Adj Close']]
largest_ohlcv =largest_ohlcv.loc[:, ['Adj Close']]
large_ohlcv,largest_ohlcv

(Price         Adj Close                                                    \
 Ticker      ADANIENT.NS       AI.PA      AIR.PA      ALV.DE BAJFINANCE.NS   
 Date                                                                        
 1962-01-02          NaN         NaN         NaN         NaN           NaN   
 1962-01-03          NaN         NaN         NaN         NaN           NaN   
 1962-01-04          NaN         NaN         NaN         NaN           NaN   
 1962-01-05          NaN         NaN         NaN         NaN           NaN   
 1962-01-08          NaN         NaN         NaN         NaN           NaN   
 ...                 ...         ...         ...         ...           ...   
 2024-05-01          NaN         NaN         NaN         NaN           NaN   
 2024-05-02  3039.600098  181.059998  153.580002  267.399994   6882.700195   
 2024-05-03  2993.250000  180.619995  154.339996  264.200012   6931.500000   
 2024-05-06  2874.649902  181.899994  156.660004  269.200012   6

In [58]:
large_ohlcv = pd.concat([large_ohlcv,
                         pd.concat({f'growth_7d': large_ohlcv['Adj Close'] / large_ohlcv['Adj Close'].shift(7)}, axis=1)
                         ], axis=1)
largest_ohlcv = pd.concat([largest_ohlcv,
                         pd.concat({f'growth_7d': largest_ohlcv['Adj Close'] / largest_ohlcv['Adj Close'].shift(7)}, axis=1)
                         ], axis=1)

large_ohlcv.head()
largest_ohlcv.head()

Adj Close                                                         \
Ticker          AAPL ACN AMZN ASML AVGO BHARTIARTL.NS BRK-B CDI.PA GOOG HDB   
Date                                                                          
1972-06-01       NaN NaN  NaN  NaN  NaN           NaN   NaN    NaN  NaN NaN   
1972-06-02       NaN NaN  NaN  NaN  NaN           NaN   NaN    NaN  NaN NaN   
1972-06-05       NaN NaN  NaN  NaN  NaN           NaN   NaN    NaN  NaN NaN   
1972-06-06       NaN NaN  NaN  NaN  NaN           NaN   NaN    NaN  NaN NaN   
1972-06-07       NaN NaN  NaN  NaN  NaN           NaN   NaN    NaN  NaN NaN   

            ... growth_7d                                                     \
Ticker      ...       NVO OR.PA RELIANCE.NS RMS.PA SAP SBIN.NS SIE.DE TCS.NS   
Date        ...                                                                
1972-06-01  ...       NaN   NaN         NaN    NaN NaN     NaN    NaN    NaN   
1972-06-02  ...       NaN   NaN         NaN    NaN NaN     NaN    NaN    NaN   
1972-06-05  ...       NaN   NaN         NaN    NaN NaN     NaN    NaN    NaN   
1972-06-06  ...       NaN   NaN         NaN    NaN NaN     NaN    NaN    NaN   
1972-06-07  ...       NaN   NaN         NaN    NaN NaN     NaN    NaN    NaN   

                    
Ticker     TTE   V  
Date                
1972-06-01 NaN NaN  
1972-06-02 NaN NaN  
1972-06-05 NaN NaN  
1972-06-06 NaN NaN  
1972-06-07 NaN NaN  

[5 rows x 66 columns]

In [59]:
count_days=len(large_ohlcv)
print(count_days)

15964


In [ ]:
large_ohlcv

Adj Close                                                    \
Ticker      ADANIENT.NS       AI.PA      AIR.PA      ALV.DE BAJFINANCE.NS   
Date                                                                        
1962-01-02          NaN         NaN         NaN         NaN           NaN   
1962-01-03          NaN         NaN         NaN         NaN           NaN   
1962-01-04          NaN         NaN         NaN         NaN           NaN   
1962-01-05          NaN         NaN         NaN         NaN           NaN   
1962-01-08          NaN         NaN         NaN         NaN           NaN   
...                 ...         ...         ...         ...           ...   
2024-04-24  3043.550049  186.139999  162.220001  266.700012   7329.149902   
2024-04-25  3115.250000  183.619995  158.520004  262.700012   7294.899902   
2024-04-26  3080.399902  185.179993  157.059998  265.500000   6731.200195   
2024-04-29  3081.199951  184.399994  155.899994  266.299988   6826.600098   
2024-04-30  3054.699951  183.880005  154.679993  266.600006   6923.549805   

                                                                 ...  \
Ticker            BUD CDI.PA        COST     DTE.DE       EL.PA  ...   
Date                                                             ...   
1962-01-02        NaN    NaN         NaN        NaN         NaN  ...   
1962-01-03        NaN    NaN         NaN        NaN         NaN  ...   
1962-01-04        NaN    NaN         NaN        NaN         NaN  ...   
1962-01-05        NaN    NaN         NaN        NaN         NaN  ...   
1962-01-08        NaN    NaN         NaN        NaN         NaN  ...   
...               ...    ...         ...        ...         ...  ...   
2024-04-24  60.259998  745.0  722.729980  21.770000  204.699997  ...   
2024-04-25  59.939999  727.5  721.859985  21.670000  201.600006  ...   
2024-04-26  60.009998  741.0  729.179993  21.850000  203.300003  ...   
2024-04-29  60.299999  735.5  726.330017  21.790001  201.600006  ...   
2024-04-30  59.660000  730.5  722.900024  21.490000  200.800003  ...   

           growth_7d                                                           \
Ticker        PRX.AS       SNY     SU.PA SUNPHARMA.NS TATAMOTORS.NS  TITAN.NS   
Date                                                                            
1962-01-02       NaN       NaN       NaN          NaN           NaN       NaN   
1962-01-03       NaN       NaN       NaN          NaN           NaN       NaN   
1962-01-04       NaN       NaN       NaN          NaN           NaN       NaN   
1962-01-05       NaN       NaN       NaN          NaN           NaN       NaN   
1962-01-08       NaN       NaN       NaN          NaN           NaN       NaN   
...              ...       ...       ...          ...           ...       ...   
2024-04-24  1.067750  1.008002  1.002593     0.964741      0.992841  1.002541   
2024-04-25  1.078215  1.077729  1.012937     0.988683      1.008511  0.979484   
2024-04-26  1.110623  1.065496  1.037470          NaN           NaN       NaN   
2024-04-29  1.099930  1.081516  1.003251     1.003429      1.030113  1.022623   
2024-04-30  1.104064  1.063283  1.032150     0.986407      1.046408  1.007353   

                                                    
Ticker          TSLA       UNH       WMT       XOM  
Date                                                
1962-01-02       NaN       NaN       NaN       NaN  
1962-01-03       NaN       NaN       NaN       NaN  
1962-01-04       NaN       NaN       NaN       NaN  
1962-01-05       NaN       NaN       NaN       NaN  
1962-01-08       NaN       NaN       NaN       NaN  
...              ...       ...       ...       ...  
2024-04-24  1.004025  1.093508  0.998999  1.011447  
2024-04-25  1.083190  1.053253  1.006183  1.022243  
2024-04-26  1.082599  1.034155  1.008550  0.994352  
2024-04-29  1.294271  0.991585  1.016537  1.009450  
2024-04-30  1.246379  0.965219  0.996976  0.986570  

[15959 rows x 66 columns]

In [ ]:
(
        large_ohlcv
        .loc[(large_ohlcv.index >= "2014-01-01") & (large_ohlcv.index <= "2023-12-31"),['growth_7d']]
        .groupby(level=0, axis=1).mean()
        .rename(columns={"growth_7d": "growth_7d_large"})
)


,growth_7d_large
Date,
2014-01-01,1.009288
2014-01-02,0.999348
2014-01-03,0.999633
2014-01-06,0.990708
2014-01-07,0.990938
...,...
2023-12-22,1.003809
2023-12-26,1.014441
2023-12-27,1.003292


Create a dataframe containing growth_7d for LARGE and LARGEST stocks

In [60]:
(
  pd.concat([
    (
        large_ohlcv
        .loc[(large_ohlcv.index >= "2014-01-01") & (large_ohlcv.index <= "2023-12-31"),['growth_7d']]
        .groupby(level=0, axis=1).mean()
        .rename(columns={"growth_7d": "growth_7d_large"})
    ),
    (
        largest_ohlcv
        .loc[(largest_ohlcv.index >= "2014-01-01") & (largest_ohlcv.index <= "2023-12-31"),['growth_7d']]
        .groupby(level=0, axis=1).mean()
        .rename(columns={"growth_7d": "growth_7d_largest"})
    )], axis=1
  )
  .loc['2014-01-01']
)

growth_7d_large      1.009288
growth_7d_largest    1.011797
Name: 2014-01-01 00:00:00, dtype: float64

Compare and extract how many times growth_7d for LARGE STOCKS is higher than for LARGEST STOCKS:

In [61]:
times_large_gt_largest = (
  pd.concat([
    (
        large_ohlcv
        .loc[(large_ohlcv.index >= "2014-01-01") & (large_ohlcv.index <= "2023-12-31"),['growth_7d']]
        .groupby(level=0, axis=1).mean()
        .rename(columns={"growth_7d": "growth_7d_large"})
    ),
    (
        largest_ohlcv
        .loc[(largest_ohlcv.index >= "2014-01-01") & (largest_ohlcv.index <= "2023-12-31"),['growth_7d']]
        .groupby(level=0, axis=1).mean()
        .rename(columns={"growth_7d": "growth_7d_largest"})
    )], axis=1
  )
  .loc[lambda df: df['growth_7d_large'] > df['growth_7d_largest']]
  .count()
)[0]

In [62]:
print("Total days: ",len(large_ohlcv.loc[(large_ohlcv.index >= "2014-01-01") & (large_ohlcv.index <= "2023-12-31")].index))
print("% days LARGE STOCKS is higher than LARGEST STOCKS: ",np.rint((times_large_gt_largest / len(large_ohlcv.loc[(large_ohlcv.index >= "2014-01-01") & (large_ohlcv.index <= "2023-12-31")].index))*100))

Total days:  2595
% days LARGE STOCKS is higher than LARGEST STOCKS:  47.0


### Question 4: Trying Another Technical Indicators strategy

**What's the total gross profit (in THOUSANDS of $) you'll get from trading on CCI (no fees assumption)?**


First, run the entire Colab to obtain the full DataFrame of data (after [Code Snippet 9]), and truncate it to the last full 10 years of data (2014-01-01 to 2023-12-31).
If you encounter any difficulties running the Colab - you can download it using this [link](https://drive.google.com/file/d/1m3Qisfs2XfWk6Sw_Uk5kHLWqwQ0q8SKb/view?usp=sharing).

Let's assume you've learned about the awesome **CCI indicator** ([Commodity Channel Index](https://www.investopedia.com/terms/c/commoditychannelindex.asp)), and decided to use only it for your operations.

You defined the "defensive" value of a high threshould of 200, and you trade only on Fridays (`Date.dt.dayofweek()==4`).

That is, every time you see that CCI is >200 for any stock (out of those 33), you'll invest $1000 (each record when CCI>200) at Adj.Close price and hold it for 1 week (5 trading days) in order to sell at the Adj. Close price.

What's the expected gross profit (no fees) that you get in THOUSANDS $ (closest integer value) over many operations in 10 years?
One operation calculations: if you invested $1000 and received $1010 in 5 days - you add $10 to gross profit, if you received $980 - add -$20 to gross profit.
You need to sum these results over all trades (460 times in 10 years).

Additional:
  * Add an approximate fees calculation over the 460 trades from this calculator https://www.degiro.ie/fees/calculator (Product:"Shares, USA and Canada;" Amount per transaction: "1000 EUR"; Transactions per year: "460")
  * are you still profitable on those trades?


In [ ]:
import pandas as pd
import pyarrow
import fastparquet

### First approach: Recreate the dataset

In [ ]:
# https://companiesmarketcap.com/usa/largest-companies-in-the-usa-by-market-cap/
US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']

# You're required to add EU_STOCKS and INDIA_STOCS
# https://companiesmarketcap.com/european-union/largest-companies-in-the-eu-by-market-cap/
EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']

# https://companiesmarketcap.com/india/largest-companies-in-india-by-market-cap/
INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']

ALL_TICKERS = US_STOCKS  + EU_STOCKS + INDIA_STOCKS
ALL_TICKERS

print(ALL_TICKERS)
print(len(ALL_TICKERS))

['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO', 'V', 'JPM', 'NVO', 'MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE', 'IDEXY', 'CDI.PA', 'RELIANCE.NS', 'TCS.NS', 'HDB', 'BHARTIARTL.NS', 'IBN', 'SBIN.NS', 'LICI.NS', 'INFY', 'ITC.NS', 'HINDUNILVR.NS', 'LT.NS']
33


Create a dataframe with the tickers defined and the OHLCV values

In [ ]:
import time

stocks_df = pd.DataFrame({'A' : []})

for i,ticker in enumerate(set(ALL_TICKERS)):
  print(i,ticker)

  # Work with stock prices
  historyPrices = yf.download(tickers = ticker,
                     period = "max",
                     #start="2014-01-01", end="2023-12-31",
                     interval = "1d")

  # generate features for historical prices, and what we want to predict
  historyPrices['Ticker'] = ticker
  historyPrices['Year']= historyPrices.index.year
  historyPrices['Month'] = historyPrices.index.month
  historyPrices['Weekday'] = historyPrices.index.weekday
  historyPrices['Date'] = historyPrices.index.date

  # historical returns
  #for i in [1,3,7]:
  i= 5
  #historyPrices[f'Adj Close {i}'] = historyPrices['Adj Close'].shift(i)
  #historyPrices['growth_'+str(i)+'d'] = historyPrices['Adj Close'] / historyPrices['Adj Close'].shift(i)
  historyPrices[f'Adj Close {i}'] = historyPrices['Adj Close'].shift(-i)
  historyPrices[f'Date {i}'] = historyPrices['Date'].shift(i)
  historyPrices['quantity'] = 1000.0/ historyPrices['Adj Close']
  historyPrices['invest'] = historyPrices['Adj Close']*historyPrices['quantity']
  historyPrices['retinvest'] = historyPrices[f'Adj Close {i}']*historyPrices['quantity']
  historyPrices['profit'] = historyPrices['retinvest']-historyPrices['invest']


  # sleep 1 sec between downloads - not to overload the API server
  time.sleep(1)


  if stocks_df.empty:
    stocks_df = historyPrices
  else:
    stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)


0 ITC.NS


[*********************100%%**********************]  1 of 1 completed


1 IDEXY


[*********************100%%**********************]  1 of 1 completed


2 NVO


...

[*********************100%%**********************]  1 of 1 completed


31 SAP


[*********************100%%**********************]  1 of 1 completed


32 MSFT


[*********************100%%**********************]  1 of 1 completed


Calculate CCI values

In [ ]:
talib_momentum_cci = talib.CCI(stocks_df.High.values, stocks_df.Low.values, stocks_df.Close.values, timeperiod=14)

In [ ]:
stocks_df['cci']=talib_momentum_cci

In [ ]:
stocks_df

,Open,High,Low,Close,Adj Close,Volume,Ticker,Year,Month,Weekday,Date,Adj Close 5,Date 5,quantity,invest,retinvest,profit,cci
0,5.550000,5.600000,5.533333,5.583333,3.255737,985500,ITC.NS,1996,1,0,1996-01-01,2.940205,None,307.150133,1000.0,903.084314,-96.915686,NaN
1,5.466666,5.566666,5.288888,5.372222,3.132634,7470000,ITC.NS,1996,1,1,1996-01-02,2.932430,None,319.220215,1000.0,936.090793,-63.909207,NaN
2,5.133333,5.254444,5.101111,5.200000,3.032208,15160500,ITC.NS,1996,1,2,1996-01-03,2.922711,None,329.792683,1000.0,963.888825,-36.111175,NaN
3,5.200000,5.332222,5.144444,5.297777,3.089223,12397500,ITC.NS,1996,1,3,1996-01-04,2.938262,None,323.705978,1000.0,951.133124,-48.866876,NaN
4,5.297777,5.277777,5.188888,5.202222,3.033504,5008500,ITC.NS,1996,1,4,1996-01-05,2.967417,None,329.651833,1000.0,978.214375,-21.785625,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221082,405.250000,406.320007,399.190002,402.250000,402.250000,19582100,MSFT,2024,4,0,2024-04-29,NaN,2024-04-22,2.486016,1000.0,NaN,NaN,-67.854481
221083,401.489990,402.160004,389.170013,389.329987,389.329987,28781400,MSFT,2024,4,1,2024-04-30,NaN,2024-04-23,2.568515,1000.0,NaN,NaN,-128.439739
221084,392.609985,401.720001,390.309998,394.940002,394.940002,23562500,MSFT,2024,5,2,2024-05-01,NaN,2024-04-24,2.532030,1000.0,NaN,NaN,-99.820469
221085,397.660004,399.929993,394.649994,397.839996,397.839996,17709400,MSFT,2024,5,3,2024-05-02,NaN,2024-04-25,2.513573,1000.0,NaN,NaN,-71.359940


In [ ]:
profit=(
    stocks_df
    #.loc[(df['Date']>='2014-01-01') & (df['Date']<='2014-20-01')&(df['Weekday']==4)]
    #.loc[(df['Date']>='2014-01-01') & (df['Date']<='2023-12-31')] #&(df['Weekday']==4)] #& (df['cci']>200)]
    [['Date','Weekday', 'Ticker','cci','Adj Close','quantity','invest','retinvest','profit']]
    .loc[(stocks_df['Date']>=date(2014,1,1)) & (stocks_df['Date']<=date(2023,12,31))]
    .loc[(stocks_df['Weekday']==4) & (stocks_df['cci']>200)] #Date.dt.dayofweek()==4
    #.loc[(df['Date'].dt.dayofweek()==4) & (df['cci']>200)]
    #[['profit']].sum()
    .sort_values(['Date','Ticker'])
)

In [ ]:
profit['profit'].sum()

1048.2598855970355

### Second approach: Reuse the data saved and shared

In [3]:
parquet_file = "stocks_df_combined_trunc_2014_2023.parquet.brotli"
df = pd.read_parquet(parquet_file)#, engine='auto')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 80762 entries, 7011 to 5342
Columns: 202 entries, Open to growth_btc_usd_365d
dtypes: datetime64[ns](3), float64(128), int32(64), int64(5), object(2)
memory usage: 105.4+ MB


In [5]:
df.head()

,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Year,Month,Weekday,...,growth_brent_oil_7d,growth_brent_oil_30d,growth_brent_oil_90d,growth_brent_oil_365d,growth_btc_usd_1d,growth_btc_usd_3d,growth_btc_usd_7d,growth_btc_usd_30d,growth_btc_usd_90d,growth_btc_usd_365d
7011,37.349998,37.400002,37.099998,37.160000,31.233059,30632200.0,MSFT,2014,2014-01-01,3,...,0.964302,0.992998,0.970030,1.158676,NaN,NaN,NaN,NaN,NaN,NaN
7012,37.200001,37.220001,36.599998,36.910000,31.022930,31134800.0,MSFT,2014,2014-01-01,4,...,0.958139,0.984707,0.961500,1.143209,NaN,NaN,NaN,NaN,NaN,NaN
7013,36.849998,36.889999,36.110001,36.130001,30.367352,43603700.0,MSFT,2014,2014-01-01,0,...,0.953798,0.998223,0.968951,1.168236,NaN,NaN,NaN,NaN,NaN,NaN
7014,36.330002,36.490002,36.209999,36.410000,30.602673,35802800.0,MSFT,2014,2014-01-01,1,...,0.958653,0.993430,0.977598,1.097648,NaN,NaN,NaN,NaN,NaN,NaN
7015,36.000000,36.139999,35.580002,35.759998,30.056356,59971700.0,MSFT,2014,2014-01-01,2,...,0.955161,0.973383,0.974977,1.100781,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
df_profits = (
    df
    [['Date','Weekday', 'Ticker','cci','Adj Close_x','growth_future_5d']]
    .assign(retinvest_fg= lambda df: df['growth_future_5d']*1000)
    .assign(profit_fg= lambda df: df['retinvest_fg']-1000)
    .loc[(df['Date']>='2014-01-01') & (df['Date']<='2023-12-31')]
    .loc[(df['Weekday']==4) & (df['cci']>200)] #Date.dt.dayofweek()==4
    .sort_values(['Date', 'Ticker'])
)

Now, we summed up all profits

In [27]:
df_fin[['profit_fg']].sum()

profit_fg    1048.262892
dtype: float64

Additional:
  * Add an approximate fees calculation over the 460 trades from this calculator https://www.degiro.ie/fees/calculator (Product:"Shares, USA and Canada;" Amount per transaction: "1000 EUR"; Transactions per year: "460")
  * are you still profitable on those trades?

WE pply the calculator and the costs are about $2,000:

![Calculator](./tax_calculator.png)


And no, it's not profitable this strategy considering the taxes.

*Initial approach to discard*

In [ ]:
ds_profit = (
    df
    [['Date','Weekday', 'Ticker','cci','Adj Close_x']]
    .assign(adj_close_5= lambda df: df['Adj Close_x'].shift(-5))
    .assign(quantity= lambda df: 1000/df['Adj Close_x'])
    .assign(invest= lambda df: df['Adj Close_x']*df['quantity'])
    .assign(retinvest= lambda df: df['adj_close_5']*df['quantity'])
    #.assign(profit= lambda df: df['retinvest']-1000.0)
    .assign(profit= lambda df: df['retinvest']-df['invest'])
    .loc[(df['Date']>='2014-01-01') & (df['Date']<='2023-12-31')]
    .loc[(df['Weekday']==4) & (df['cci']>200)] #Date.dt.dayofweek()==4
    #.loc[(df['Date'].dt.dayofweek()==4) & (df['cci']>200)]
    #[['profit']].sum()
    .sort_values(['Date', 'Ticker'])
)

In [ ]:
ds_profit['profit'].sum()

162.9540060662083

In [ ]:
ds_profit

,Date,Weekday,Ticker,cci,Adj Close_x,adj_close_5,quantity,invest,retinvest,profit
3733,2014-01-10,4,INFY,209.021780,5.770794,5.846446,173.286382,1000.0,1013.109400,13.109400
1469,2014-01-17,4,V,350.030856,54.026161,50.312408,18.509551,1000.0,931.260103,-68.739897
5700,2014-01-31,4,CDI.PA,264.157424,108.007111,109.996895,9.258650,1000.0,1018.422715,18.422715
8262,2014-01-31,4,NVO,230.180683,16.221884,17.522247,61.645122,1000.0,1080.161069,80.161069
3790,2014-02-14,4,NVDA,207.361756,4.224034,4.459884,236.740528,1000.0,1055.835333,55.835333
...,...,...,...,...,...,...,...,...,...,...
6156,2023-12-08,4,MC.PA,203.110749,732.143066,741.355591,1.365853,1000.0,1012.582957,12.582957
6691,2023-12-15,4,AMZN,206.810139,149.970001,153.419998,6.668000,1000.0,1023.004580,23.004580
6233,2023-12-15,4,INFY,286.746964,18.670000,18.760000,53.561864,1000.0,1004.820576,4.820576
5301,2023-12-15,4,TCS.NS,203.551619,3843.191895,3806.362549,0.260200,1000.0,990.416990,-9.583010
